---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    train.csv - the training set (all tickets issued 2004-2011)
    test.csv - the test set (all tickets issued 2012-2016)
    addresses.csv & latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32

In [45]:
import pandas as pd
import numpy as np

from sklearn.metrics import roc_auc_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV


class BlightDataset():
    
    def __init__(self, dataname = 'train.csv', 
                 verbose = False,
                 initial_features = None, X = None):
        
        self.verbose=verbose
        
        self._df = None
        if not (dataname is None):
            self._raw_df = pd.read_csv(dataname, encoding='latin1')
            
            # data frame that will be modified
            self.df = self._raw_df.copy()
        
  #      else if not (X is None):
            # do filtering
            
   #     else:
            
        # features used to fit the model    
        if initial_features is None:
            initial_features = ['late_fee',
                         # 'agency_name',
                         #' discount_amount',
                         # 'zip_code',
                         # 'violation_code',
                         # 'hearing_date', 
                         # 'ticket_issued_date',
                         'fine_amount',
                         'disposition']
        
        self.initial_features = initial_features
        self.features         = initial_features.copy()     
        self.target = 'compliance'
        
        return
    
    @property
    def raw_df(self):
        """
        Original dataframe of associated dataset.
        Read-only.
        """
        return self._raw_df.copy()
    
    
    def clean_data(self, df = None, 
                         remove_null_compliance=True):
        """
        This goes through and removes all unneeded data
        and cleans features.
        
        remove_null_compliance : if true, removes null compliance
                                 values from dataset
                                 if false, treats these as compliant
                                
        """
        
            
        # can optionally pass a df to be filtered
        # otherwise, uses self
        df_none = False
        if df is None:
            df = self.df.copy()
            df_none = True
        else:
            df = df.copy()
            
        training_data = self.target in df.columns

        # zip code
        #         convert < 5 to Null values
        #         convert > 5 to just first 5 digits
        df['zip_code'] = [str(x) if len(str(x)) == 5 else str(x)[:5] if len(str(x)) > 5 else 'nan' for x in df['zip_code']]        
        df['zip_code'] = [x if x.isdecimal() else 'nan' for x in df['zip_code']]
        df['zip_code'] = df['zip_code'].astype(str).replace('nan','0').astype(float)
        
        
        # violation code
        #if 'violation_code' in self.initial_features:
        #    df['violation_code'] = self.violation_code_mapper(df['violation_code'].astype('str'))
        
        # agency name
        if 'agency_name' in self.initial_features:
            df = self.agency_name_encoding(df,append_features=training_data)
        
        # hearing date
        # convert to year with decimals
        df['hearing_date'] = df['hearing_date'].astype(str)
        # date as fraction of month conversion is not strictly correct here (assumes 31 days each month)
        # but the effect does not matter vs. a more accurate decimal
        df['hearing_date']  = np.array([float(x[:4]) + float(x[5:7])/12.0 + float(x[9:11])/31.0 if x != 'nan' else 0.0 for x in df['hearing_date']])
        
        
        df['ticket_issued_date'] = df['ticket_issued_date'].astype(str)
        # date as fraction of month conversion is not strictly correct here (assumes 31 days each month)
        # but the effect does not matter vs. a more accurate decimal
        df['ticket_issued_date']  = np.array([float(x[:4]) + float(x[5:7])/12.0 + float(x[9:11])/31.0 if x != 'nan' else 0.0 for x in df['ticket_issued_date']])
               
        if 'hearing_date' in self.initial_features and 'ticket_issued_date' in self.initial_features:
            df['hearing_ticket_diff'] = df['hearing_date'] - df['ticket_issued_date']
            if training_data:
                self.features.append('hearing_ticket_diff')

        #
        # just keep this the same for now,
        # but it MIGHT be good to 
        #       
        df['judgment_amount'] = df['judgment_amount'].astype(float).fillna(-1.0)
        
        #
        #  Disposition - 
        #     Strong correlation for some of these categories, but group
        #     the similar ones together to reduce number of new features
        #     Not responsibles are all the same, pending are the same,
        #     responsible_ad behave the same
        #
        #
        if 'disposition' in self.initial_features:
            df['disposition'] = ['Not Responsible' if (('Not Responsible' in x) or ('Not responsible' in x)) else x for x in df['disposition'].astype(str)]
            df['disposition'] = ['Pending' if 'PENDING' in x else x for x in df['disposition']]
            df['disposition'] = ['Responsible AD' if (('Responsible' in x) and (('Admission' in x) or ('Determination' in x))) else x for x in df['disposition']]
            df = self.disposition_encoding(df, append_features=training_data)
        
        
     
        #
        # Processing on target field
        #
        if training_data:
            # df[self.target_field] = df[self.target_field].astype(int)
            if not remove_null_compliance:
                df[self.target].fillna(1.0, inplace=True)
                         
        if training_data:
            df = self.remove_data_for_training(df, remove_null_compliance=remove_null_compliance)
        
        # print(df.columns)

        if df_none:
            self.df = df
                
        
        #for k in X.columns:
        #    print(k, np.sum(X[k].isnull()), X[k][X[k].isnull()])

        
        if training_data:
                    
            self.y_train = self.df[self.target]
            self.X_train = self.df[self.features]
            
            return self.X_train, self.y_train
        
        else:
            
            # for columns NOT here, need to fill with zeros
            for xname in self.X_train.columns:
                if not (xname in df.columns):
                    df[xname] = np.zeros(len(df))
                    
            X = df[self.features]
            
            return X
        
        
    def train_model(self, X_train, y_train, grid_search=False):
        """
        Model training.
        """
        # try with defaults for now!
        
        self.X_train = X_train
        self.y_train = y_train
        
        
        if grid_search:
            parameters = {
                #"loss":["deviance"],
                "learning_rate": [0.01, 0.1, 0.5],
                #"min_samples_split": np.linspace(0.1, 2, 4),
               # "min_samples_leaf": np.linspace(0.1, 1, 4),
                "max_depth":[3,5,8],
                #"max_features":["log2","sqrt"],
                #"criterion": ["friedman_mse",  "mae"],
                #"subsample":[0.5, 0.75, 0.9, 1.0],
                "n_estimators":[10, 100, 500]
                }

            grid_clf = GridSearchCV(GradientBoostingClassifier(), parameters, 
                                    cv=3, n_jobs=4, scoring='roc_auc')
            
            grid_clf.fit(X_train, y_train)

            if self.verbose:
                print(grid_clf.best_params_)
            
            model_parameters = grid_clf.best_params_
        else:
            model_parameters = {'learning_rate':0.1, 
                                'max_depth' : 5,
                                'n_estimators' : 100}
            model_parameters = {'learning_rate':0.1, 
                                'max_depth' : 3,
                                'n_estimators' : 100}            
        
        clf = GradientBoostingClassifier(**model_parameters)
        clf.fit(X_train, y_train)
        
        self.clf = clf
        
        
        self.train_auc      = roc_auc_score(y_train, self.clf.predict_proba(X_train)[:,1])        
        self.train_accuracy = self.clf.score(X_train, y_train)
        
        if self.verbose:
            print("Classifier Training AOC: %5.5f"%(self.train_auc))
            print("Classifier Training Accuracy: %5.5f"%(self.train_accuracy))
        
        #return self.train_auc, self.train_accuracy        
        return        
        
    def train_dummy(self, X_train, y_train):
        """
        Dummy classifier
        """
        from sklearn.dummy import DummyClassifier

        dummy_majority = DummyClassifier(strategy = 'most_frequent').fit(X_train, y_train)
        y_dummy_predictions = dummy_majority.predict(X_train)

        self.dummy_clf = dummy_majority
        
        self.dummy_auc      = roc_auc_score(y_train, self.dummy_clf.predict_proba(X_train)[:,1])        
        self.dummy_accuracy = self.dummy_clf.score(X_train, y_train)
        
        if self.verbose:
            print("Dummy Classifier Training AOC: %5.5f"%(self.dummy_auc))
            print("Dummy Classifier Training Accuracy: %5.5f"%(self.dummy_accuracy))
        
        return self.dummy_auc, self.dummy_accuracy
                    
    def remove_data_for_training(self, df = None, 
                                       remove_null_compliance=False):
        """
        All editing is done in-place on dataframe
        """
             
        df_none = False
        if df is None:
            df_none = True
            df = self.df.copy()
        #
        # Now remove rows
        #
        mask = np.ones(len(df)).astype(bool)
        
        if remove_null_compliance and 'compliance' in df.columns:
            mask = mask * df['compliance'].notnull()
        
        
        df = df[mask]
        
        if df_none:
            self.df = df
            return 
        else:
            return df
        
    def reset_df(self):
        """
        Reset any processing on data frame and reload original 
        data.
        """
        
        self.df = self._raw_df.copy()
        
        return
    
    
    def violation_code_mapper(self, vc_to_map):
        """
        Given a list of violation codes, maps the categorical variables
        to integers given what is known ONLY in the training set as categories
        with suffient data to say anything. Everything else is lumped into a 'null'
        violcation code category (e.g category = 0). Taking this (at the moment)
        to be all categories in the training data that have > 10 counts.
        
        
        Returns a list of integers for each violation code.
        
        """    
        
        vc_value_counts = self._raw_df['violation_code'].astype(str).value_counts()
        vc_labels = vc_value_counts[ vc_value_counts > 10].index
        
        # create a dictionary mapping these
        vc_dictionary = dict(zip(vc_labels, np.arange(1,len(vc_labels)+1)))
        
        # now map input to integers
        vc_integers = np.array([vc_dictionary[x] if x in vc_dictionary.keys() else 0 for x in vc_to_map])
        
        return vc_integers
    
    def agency_name_encoding(self, df = None, append_features=True):
        """
        
        """
        df_none = False
        if df is None:
            df_none = True
            df = self.df.copy()
        
        # do one hot codying of agency names
        # as new features, and add these to the features list
        # --- remove agency_name
        dummy_df = pd.get_dummies(df['agency_name'].astype('str'), prefix='agency_name')
        df = pd.concat([df, dummy_df], axis=1)
        
        
        if append_features:
            [self.features.append(x) for x in dummy_df.columns if not (x in self.features)];
        
        if 'agency_name' in self.features:
            self.features.remove('agency_name')
        
        #for x in dummy_df.columns:
        
        #an_value_counts = self._raw_df['agency_name'].astype(str).value_counts()
        #an_labels = an_value_counts.index
        #
        # create a dictionary mapping these
        #an_dictionary = dict(zip(an_labels, np.arange(1,len(an_labels)+1)))
        
        # now map input to integers
        #an_integers = np.array([an_dictionary[x] if x in an_dictionary.keys() else 0 for x in an_to_map])
        
        if df_none:
            self.df = df
            return
        else:
            return df
    
    def disposition_encoding(self, df = None, append_features=True):
        """
        
        """
        
        df_none = False
        if df is None:
            df_none = True
            df = self.df.copy()
        
        # do one hot codying of agency names
        # as new features, and add these to the features list
        dummy_df = pd.get_dummies(df['disposition'].astype('str'), prefix='disposition')
        df = pd.concat([df, dummy_df], axis=1)
        if append_features:
            [self.features.append(x) for x in dummy_df.columns if not (x in self.features)];
        
        if 'disposition' in self.features:
            self.features.remove('disposition')

        if df_none:
            self.df = df
            return
        else:
            return  df



def blight_model():
    bd               = BlightDataset(initial_features = [
                                                     'discount_amount',
                                                     'disposition',
                                                     'hearing_date',
                                                     'ticket_issued_date',
                                                     'fine_amount',
                                                     'agency_name',
                                                     'zip_code'
                                             ], 
                                             verbose=False)
    X_train, y_train = bd.clean_data()
    bd.train_model(X_train, y_train, True)

    test_df        = pd.read_csv('test.csv', encoding='latin1')
    X_test         = bd.clean_data(test_df)
    
    result = pd.Series(bd.clf.predict_proba(X_test)[:,1], index = test_df.ticket_id.values)
    
    return result # Your answer here

In [44]:
blight_model()

/home/aemerick/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3325: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/home/aemerick/anaconda3/lib/python3.7/site-packages/pandas/core/computation/expressions.py:183: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  .format(op=op_str, alt_op=unsupported[op_str]))


284932    0.186791
285362    0.028019
285361    0.145130
285338    0.143120
285346    0.131134
285345    0.202357
285347    0.055830
285342    0.671599
285530    0.038342
284989    0.032997
285344    0.055830
285343    0.016592
285340    0.016592
285341    0.055830
285349    0.131134
285348    0.202357
284991    0.032997
285532    0.051425
285406    0.036356
285001    0.028815
285006    0.023567
285405    0.016602
285337    0.045418
285496    0.102585
285497    0.242385
285378    0.024675
285589    0.035037
285585    0.196239
285501    0.231366
285581    0.024070
            ...   
376367    0.014008
376366    0.044550
376362    0.138629
376363    0.141220
376365    0.014008
376364    0.044550
376228    0.044550
376265    0.044550
376286    0.345587
376320    0.044550
376314    0.044550
376327    0.345587
376385    0.345587
376435    0.342030
376370    0.345587
376434    0.291061
376459    0.513401
376478    0.034858
376473    0.044550
376484    0.017335
376482    0.014944
376480    0.